In [1]:
import os
import findspark
findspark.find()
findspark.init(os.environ.get("SPARK_HOME"))

import sys
sys.path.append("/Users/minjungchoi/spark/spark-2.4.0-bin-hadoop2.7/python/pyspark")

from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, collect_list,udf,concat, lit


conf = SparkConf().setAppName("comstat-test").set("spark.yarn.driver.memoryOverhead", "2048")   \
         .set("spark.yarn.executor.memoryOverhead", "2048")   \
         .set("spark.default.parallelism", "116") \
         .set("spark.shuffle.compress", "true") \
         .set("spark.io.compression.codec", "snappy")

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

In [12]:
sampleData = [
	('A','WD','1','Z001','S001'),
	('A','WD','1','Z002','S020'),
	('A','WD','2','Z005','S100'),
	('A','WE','3','Z001','S001'),
	('A','WE','2','Z002','S000'),
	('A','WE','1','Z001','S001'),
	('A','WD','3','Z001','S001'),
	('A','WD','4','Z001','S002'),
	('A','WD','4','Z002','S030'),
	('A','WD','3','Z003','S009'),
	('A','WD','1','Z001','S002'),
	('A','WD','2','Z002','S030'),
	('A','WD','3','Z001','S001'),
	('A','WD','4','Z003','S003'),
	('A','WD','4','Z003','S005'),
	('A','WD','4','Z001','S001'),
	('A','WD','3','Z005','S006'),
	('A','WE','2','Z006','S007'),
	('A','WE','3','Z001','S002'),
	('B','WD','1','Z001','S001'),
	('B','WD','1','Z002','S020'),
	('B','WD','2','Z005','S100'),
	('B','WE','3','Z001','S001'),
	('B','WE','2','Z002','S000'),
	('B','WE','1','Z001','S001'),
	('B','WD','3','Z001','S001'),
	('C','WD','4','Z001','S002'),
	('C','WD','4','Z002','S030'),
	('C','WD','3','Z003','S009'),
	('C','WD','1','Z001','S002'),
	('D','WD','2','Z002','S030'),
	('D','WD','3','Z001','S001'),
	('D','WD','4','Z003','S003'),
	('D','WD','4','Z003','S005'),
	('D','WD','4','Z001','S001'),
	('D','WD','3','Z005','S006'),
	('D','WE','2','Z006','S007'),
	('D','WE','3','Z001','S002'),
	('E','WE','1','Z003','S001'),
    ]

    
field = [
	StructField('CID',StringType(), True),\
	StructField('WEEKDAY', StringType(), True),\
	StructField('TIMESEG', StringType(), True),\
	StructField('LOCATION', StringType(), True),\
 	StructField('SHOP', StringType(), True)]

schema = StructType(field)


sampleRDD = sc.parallelize(sampleData)
sampleDF = spark.createDataFrame(sampleRDD,schema)

print('Table')
print(sampleDF.show())


Table
+---+-------+-------+--------+----+
|CID|WEEKDAY|TIMESEG|LOCATION|SHOP|
+---+-------+-------+--------+----+
|  A|     WD|      1|    Z001|S001|
|  A|     WD|      1|    Z002|S020|
|  A|     WD|      2|    Z005|S100|
|  A|     WE|      3|    Z001|S001|
|  A|     WE|      2|    Z002|S000|
|  A|     WE|      1|    Z001|S001|
|  A|     WD|      3|    Z001|S001|
|  A|     WD|      4|    Z001|S002|
|  A|     WD|      4|    Z002|S030|
|  A|     WD|      3|    Z003|S009|
|  A|     WD|      1|    Z001|S002|
|  A|     WD|      2|    Z002|S030|
|  A|     WD|      3|    Z001|S001|
|  A|     WD|      4|    Z003|S003|
|  A|     WD|      4|    Z003|S005|
|  A|     WD|      4|    Z001|S001|
|  A|     WD|      3|    Z005|S006|
|  A|     WE|      2|    Z006|S007|
|  A|     WE|      3|    Z001|S002|
|  B|     WD|      1|    Z001|S001|
+---+-------+-------+--------+----+
only showing top 20 rows

None


In [ ]:
# cidByShopMap = sampleDF.rdd.map(lambda x : (x['CID'] +':'+ x['SHOP'], 1))

# cidByShop = cidByShopMap.reduceByKey(lambda x,y : x+y)

In [ ]:
# cidByShop.take(20)

In [5]:
#%%
getTop3List = udf(lambda x:x[0:3],StringType())

In [13]:

#%%
#cross tab으로 보기
sampleDF.crosstab('CID', 'SHOP').show()

+--------+----+----+----+----+----+----+----+----+----+----+----+
|CID_SHOP|S000|S001|S002|S003|S005|S006|S007|S009|S020|S030|S100|
+--------+----+----+----+----+----+----+----+----+----+----+----+
|       E|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|
|       A|   1|   6|   3|   1|   1|   1|   1|   1|   1|   2|   1|
|       B|   1|   4|   0|   0|   0|   0|   0|   0|   1|   0|   1|
|       C|   0|   0|   2|   0|   0|   0|   0|   1|   0|   1|   0|
|       D|   0|   2|   1|   1|   1|   1|   1|   0|   0|   1|   0|
+--------+----+----+----+----+----+----+----+----+----+----+----+



In [14]:
#%%
#고객별 최다 방문 SHOP TOP3
BySHOP = sampleDF.groupby('CID', 'SHOP').count().orderBy('CID', 'count',ascending=False)

BySHOP.show()

BySHOP = BySHOP.groupby('CID').agg(collect_list('SHOP').alias('SHOP3'))\
          .withColumn('SHOP3', getTop3List('SHOP3'))

BySHOP.show()

+---+----+-----+
|CID|SHOP|count|
+---+----+-----+
|  E|S001|    1|
|  D|S001|    2|
|  D|S002|    1|
|  D|S030|    1|
|  D|S006|    1|
|  D|S005|    1|
|  D|S003|    1|
|  D|S007|    1|
|  C|S002|    2|
|  C|S030|    1|
|  C|S009|    1|
|  B|S001|    4|
|  B|S100|    1|
|  B|S020|    1|
|  B|S000|    1|
|  A|S001|    6|
|  A|S002|    3|
|  A|S030|    2|
|  A|S005|    1|
|  A|S007|    1|
+---+----+-----+
only showing top 20 rows

+---+------------------+
|CID|             SHOP3|
+---+------------------+
|  E|            [S001]|
|  B|[S001, S020, S100]|
|  D|[S001, S003, S007]|
|  C|[S002, S009, S030]|
|  A|[S001, S002, S030]|
+---+------------------+



In [15]:
#%%
#고객별 최다 방문 지역 TOP3
ByLOCATION = sampleDF.groupby('CID', 'LOCATION').count().orderBy('CID', 'count',ascending=False)

ByLOCATION.show()

ByLOCATION = ByLOCATION.groupby('CID').agg(collect_list('LOCATION').alias('Location3'))\
          .withColumn('Location3', getTop3List('Location3'))

ByLOCATION.show()

+---+--------+-----+
|CID|LOCATION|count|
+---+--------+-----+
|  E|    Z003|    1|
|  D|    Z001|    3|
|  D|    Z003|    2|
|  D|    Z005|    1|
|  D|    Z002|    1|
|  D|    Z006|    1|
|  C|    Z001|    2|
|  C|    Z002|    1|
|  C|    Z003|    1|
|  B|    Z001|    4|
|  B|    Z002|    2|
|  B|    Z005|    1|
|  A|    Z001|    9|
|  A|    Z002|    4|
|  A|    Z003|    3|
|  A|    Z005|    2|
|  A|    Z006|    1|
+---+--------+-----+

+---+------------------+
|CID|         Location3|
+---+------------------+
|  E|            [Z003]|
|  B|[Z001, Z002, Z005]|
|  D|[Z001, Z003, Z006]|
|  C|[Z001, Z003, Z002]|
|  A|[Z001, Z002, Z003]|
+---+------------------+



In [16]:
#%%
#고객별 최다 방문 주중/주말 방문 SHOP TOP3
ByWEEKDAY_SHOP = sampleDF.groupby('CID', 'WEEKDAY', 'SHOP').count().orderBy('CID', 'count',ascending=False)\
                         .select('CID', (concat(col('WEEKDAY') ,lit('|'),col('SHOP')).alias('WEEKDAY_SHOP')), 'count')
ByWEEKDAY_SHOP.show()
ByWEEKDAY_SHOP = ByWEEKDAY_SHOP.groupby('CID').agg(collect_list('WEEKDAY_SHOP').alias('WEEKDAY_SHOP3'))\
          .withColumn('WEEKDAY_SHOP3', getTop3List('WEEKDAY_SHOP3'))


ByWEEKDAY_SHOP.show()

+---+------------+-----+
|CID|WEEKDAY_SHOP|count|
+---+------------+-----+
|  E|     WE|S001|    1|
|  D|     WD|S001|    2|
|  D|     WE|S002|    1|
|  D|     WD|S005|    1|
|  D|     WD|S030|    1|
|  D|     WE|S007|    1|
|  D|     WD|S006|    1|
|  D|     WD|S003|    1|
|  C|     WD|S002|    2|
|  C|     WD|S009|    1|
|  C|     WD|S030|    1|
|  B|     WE|S001|    2|
|  B|     WD|S001|    2|
|  B|     WD|S020|    1|
|  B|     WD|S100|    1|
|  B|     WE|S000|    1|
|  A|     WD|S001|    4|
|  A|     WD|S002|    2|
|  A|     WE|S001|    2|
|  A|     WD|S030|    2|
+---+------------+-----+
only showing top 20 rows

+---+--------------------+
|CID|       WEEKDAY_SHOP3|
+---+--------------------+
|  E|           [WE|S001]|
|  B|[WD|S001, WE|S001...|
|  D|[WD|S001, WD|S005...|
|  C|[WD|S002, WD|S030...|
|  A|[WD|S001, WD|S002...|
+---+--------------------+



In [17]:
#%%
#고객별로 한테이블로 모으자 (굳이 모을 필요는 없고 각자 테이블로 가지고 있는게 나을 걸?)
#별도의 고객리스트가 없으면 트랜잭션에서 빼내자
users = sampleDF.select('CID').distinct()

totalSummary = users.join(BySHOP, BySHOP.CID == users.CID, how='left').drop(BySHOP.CID)

totalSummary = totalSummary.join(ByLOCATION, ByLOCATION.CID == totalSummary.CID, how='left').drop(ByLOCATION.CID)
totalSummary = totalSummary.join(ByWEEKDAY_SHOP, ByWEEKDAY_SHOP.CID == totalSummary.CID, how='left').drop(ByWEEKDAY_SHOP.CID)
totalSummary.show()


+---+------------------+------------------+--------------------+
|CID|             SHOP3|         Location3|       WEEKDAY_SHOP3|
+---+------------------+------------------+--------------------+
|  E|            [S001]|            [Z003]|           [WE|S001]|
|  B|[S001, S020, S100]|[Z001, Z002, Z005]|[WD|S001, WE|S001...|
|  D|[S001, S003, S007]|[Z001, Z003, Z006]|[WD|S001, WD|S005...|
|  C|[S002, S009, S030]|[Z001, Z003, Z002]|[WD|S002, WD|S030...|
|  A|[S001, S002, S030]|[Z001, Z002, Z003]|[WD|S001, WD|S002...|
+---+------------------+------------------+--------------------+



In [18]:
#%%
## map reduce로 그냥 구현
cidByShopMap = sampleDF.rdd.map(lambda x : (x['CID'] +':'+ x['SHOP'], 1))

cidByShop = cidByShopMap.reduceByKey(lambda x,y : x+y)

print('Customer by shop')
print(cidByShop.take(50))

print('Customer by shop TOP 3')
print( cidByShop.sortBy(lambda x: x[1], ascending= False).take(3) )

Customer by shop
[('C:S002', 2), ('A:S030', 2), ('A:S002', 3), ('A:S007', 1), ('D:S002', 1), ('B:S100', 1), ('D:S001', 2), ('D:S006', 1), ('D:S007', 1), ('A:S005', 1), ('A:S009', 1), ('A:S020', 1), ('D:S030', 1), ('D:S005', 1), ('A:S100', 1), ('B:S020', 1), ('D:S003', 1), ('B:S000', 1), ('A:S006', 1), ('A:S001', 6), ('B:S001', 4), ('A:S003', 1), ('C:S030', 1), ('C:S009', 1), ('E:S001', 1), ('A:S000', 1)]
Customer by shop TOP 3
[('A:S001', 6), ('B:S001', 4), ('A:S002', 3)]


In [ ]:
#%%
sc.stop()